Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


Mounted at /content/drive


In [28]:
#Raw Data Location

temp="/content/drive/MyDrive/P2M/Data/temperature.csv"
mvt="/content/drive/MyDrive/P2M/Data/mvt.csv"

tempmod="/content/drive/MyDrive/P2M/Data/temperaturemod1.csv"
mvtmod="/content/drive/MyDrive/P2M/Data/mvtmod1.csv"

In [29]:
data=pd.read_csv(temp,header=None)
data1=pd.read_csv(mvt,header=None)

# Data Cleaning

In [30]:
headerlist=["Id", "id_box", "Emp_box","indicateur","temp","Rest4","Rest3","time_stamp1","time_stamp2",
            "Rest7","Rest8","Rest9","Rest10","Rest11","Rest12","Rest13","Rest14","Rest15","Rest16","Rest17","ClimName"]
data.to_csv(tempmod, header=headerlist, index=False)
data = pd.read_csv(tempmod)

In [31]:
data.shape

(4346120, 21)

In [32]:
data = data.drop(columns=["Id","Rest3","Rest4","time_stamp2","Rest7","Rest8","Rest9","Rest10","Rest11","Rest12","Rest13","Rest14","Rest15","Rest16","Rest17"],axis = 1)

headerlist2=["Id_box","Emp_box","Indicateur","Temp","Time_stamp","ClimName"]
data.to_csv(tempmod, header=headerlist2, index=False)
data.head()

,id_box,Emp_box,indicateur,temp,time_stamp1,ClimName
0,542,168,67,29,2021-08-10 00:00:04,Clim iot 04
1,542,168,67,30,2021-08-10 00:00:12,Clim iot 04
2,542,168,67,29,2021-08-10 00:00:21,Clim iot 04
3,542,168,67,29,2021-08-10 00:00:31,Clim iot 04
4,542,168,67,29,2021-08-10 00:00:35,Clim iot 04


In [33]:
headerlist1=["Id", "id_box", "Emp_box","indicateur","mvt","Rest4","Rest3","time_stamp1","time_stamp2","Rest7","Rest8",
             "Rest9","Rest10","Rest11","Rest12","Rest13","Rest14","Rest15","Rest16","Rest17","ClimName"]
data1.to_csv(mvtmod, header=headerlist1, index=False)
data1 = pd.read_csv(mvtmod)

In [34]:
data1 = data1.drop(columns=["Id","Rest3","Rest4","Rest7","Rest8","Rest9","Rest10","Rest11","Rest12","Rest13","Rest14","Rest15","Rest16","Rest17","time_stamp2"])

headerlist2=["Id_box","Emp_box","Indicateur","Mvt","Time_stamp","ClimName"]
data1.to_csv(mvtmod, header=headerlist2, index=False)
data1.head()

,id_box,Emp_box,indicateur,mvt,time_stamp1,ClimName
0,543,168,207,1,2021-08-10 00:00:08,Clim iot 04
1,543,168,207,1,2021-08-10 00:00:14,Clim iot 04
2,543,168,207,1,2021-08-10 00:00:25,Clim iot 04
3,543,168,207,1,2021-08-10 00:00:40,Clim iot 04
4,543,168,207,1,2021-08-10 00:00:40,Clim iot 04


# Temperature scrapping

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup as bs
from itertools import zip_longest 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
scrap="/content/drive/MyDrive/P2M/Data/ScrapTemp.csv"

In [ ]:
mois={8:"aout",9:"septembre",10:"octobre",11:"novembre",12:"decembre"}
def formatage_temp(temp):
  temp=temp.replace('°','')
  if(temp.isnumeric()):
    temp=int(temp)
  else:
    temp=int(0)
  return temp
def formatage_date(ch,k):
  l=ch.split()
  if k<10:
    k="0"+str(k)
  ch1=l[3]+"-"+str(k)+"-"+l[1]+" "
  return ch1
tmps=[]
heures=[]  
for k in range (8,13):
  y=mois[k]
  print(y)
  if(k%2 == 0):
    jours=31
  else:
    jours=30
  x=0
  temp=[]
  heure=[]
  for i in range (0,jours):
    x=x+1
    url1="https://fr.tutiempo.net/enregistrements/dttb/"+str(x)+"-"+mois[k]+"-2021.html"
    result=requests.get(url1)
    src=result.content
    soup = bs(src,"lxml")
    date=soup.find_all("th",{"class":"ndia"})
    temps=soup.find_all("td",{"class":"t Temp"}) 
    for j in range (len(temps)):
      ch1=formatage_date(date[0].text,k)
      #2021-08-10 00:00:04
      if j<10:
        ch=ch1+"   "+"0"+str(j)+":00:00"
      else:
        ch=ch1+"   "+str(j)+":00:00"
      heure.append(ch)
      temp.append(formatage_temp(temps[j].text))
  tmps=tmps+temp
  heures=heures+heure

file_list=[heures,tmps]
exported=zip_longest(*file_list)

with open(scrap,"w")as myfile:
  wr = csv.writer(myfile)
  wr.writerow(["Time_stamp","TempExt"])
  wr.writerows(exported)
  

#Scrapped data formatting and cleaning

In [ ]:
fil=pd.read_csv(scrap)
fil.head()

In [ ]:
fil.describe()

In [ ]:
fil.boxplot('TempExt')

In [ ]:
fil.loc[fil['TempExt']==40]

In [ ]:
fil.columns

In [ ]:
def check_space(string):
    count = 0
    for i in range(0, len(string)):
        if string[i] == " ":
            count += 1
         
    return count
#check the number of spaces in column time_stamp    
string = fil.Time_stamp[1]
print(string)
print("number of spaces ",check_space(string))

In [ ]:
fil[['Date','Time']]=fil.Time_stamp.str.split("    ",expand=True)
fil = fil.drop(columns="Time_stamp")
fil.columns

In [ ]:
# minutes and seconds data frame
def create_timestamp(hs):
  timeStamp=[]
  for m in range(0,60):
    for s in range(1,60):
      ss=str(s)
      ms=str(m)
      if s<10 :
        ss="0"+str(s)
      if m<10 :
        ms="0"+str(m)
      timeStamp.append(hs+":"+ms +":"+ ss) 
  return(timeStamp)
compt=0
last=len(fil)
for i in range(0,last):
  line=fil.iloc[i]
  #Build the Dataframe to concat
  hs=line.Time[:2]
  time=create_timestamp(hs)
  date=[line.Date]*len(time)
  temp=[line.TempExt]*len(time)
  #'Time_stamp', 'TempExt', 'Date', 'Time'
  dict={'TempExt':temp, 'Date':date, 'Time':time}
  toConcat=pd.DataFrame(dict)
  #Last step concat
  fil=pd.concat([fil,toConcat])


In [ ]:
fil.TempExt.notna().sum()

In [ ]:
fil=fil.sort_values(by=['Date','Time'])

In [ ]:
fil=fil.reset_index()
fil=fil.drop(columns="index")
fil.columns

In [ ]:
fil.head()

In [ ]:
fil.describe()

Test for duplicates

In [ ]:
fil.duplicated(subset=['Time','Date']).any()

In [ ]:
ScrapedDataPath="/content/drive/MyDrive/P2M/Data/ScrapedData.csv"
headerlist=['TempExt', 'Date', 'Time']
fil.to_csv(ScrapedDataPath, header=headerlist, index=False)

In [ ]:
final=pd.read_csv(ScrapedDataPath)
final.head()